In [2]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.naive_bayes import GaussianNB

def clean(x):
    return len(x.strip()) > 1 and not x.startswith("#") and not any(c.isalpha() for c in x) # Bỏ dòng trống và dòng bắt đầu bằng "#"

def readData(path):
    with open(path, encoding='utf-8') as f:
        content = f.readlines()
        content = [x.strip() for x in content if clean(x)]  # Xóa khoảng trắng thừa và lọc dòng lỗi
    
    data = []
    label = []

    for line in content:
        a = line.split(",")
        if len(a) < 11:  # Đảm bảo đủ 11 cột (mã số, nhãn, 9 đặc trưng)
            print(f"Warning: Bỏ qua dòng không hợp lệ: {line}")
            continue  # Bỏ qua dòng không đúng định dạng
        
        try:
            data.append(list(map(int, a[2:])))  # Chuyển tất cả các giá trị đặc trưng sang số nguyên
            label.append(int(a[1]))  # Lấy nhãn
        except ValueError as e:
            print(f"Warning: Lỗi chuyển đổi dữ liệu ở dòng: {line}, lỗi: {e}")
            continue  # Bỏ qua nếu có lỗi

    return np.array(data, dtype=int), np.array(label, dtype=int)

# Đọc dữ liệu
data, label = readData("D:/Hoc may/datacum.txt") 

# Kiểm tra kích thước dữ liệu để tránh lỗi
print("Number of samples read:", len(data))
print("Shape of the dataset:", data.shape)
print("Number of labels:", len(label))

# Kiểm tra nếu dữ liệu có đủ để chia thành test và train
if len(data) <= 120:
    raise ValueError("Dữ liệu không đủ để chia thành tập huấn luyện và kiểm tra!")

# Khởi tạo tập test và train
data_test = np.zeros((120, 9), dtype=int)
data2_test_count = 0
data4_test_count = 0
data_test_count = 0
data_train_count = 0

X_train = np.zeros((len(data) - 120, 9), dtype=int)
Y_train = np.zeros((len(data) - 120,), dtype=int)
Y_test = np.zeros((120,), dtype=int)

# Chia dữ liệu thành train và test
for i in range(len(data)):
    if label[i] == 2 and data2_test_count < 80:
        data2_test_count += 1
        data_test[data_test_count] = data[i]
        Y_test[data_test_count] = label[i]
        data_test_count += 1
    elif label[i] == 4 and data4_test_count < 40:
        data4_test_count += 1
        data_test[data_test_count] = data[i]
        Y_test[data_test_count] = label[i]
        data_test_count += 1
    else:
        X_train[data_train_count] = data[i]
        Y_train[data_train_count] = label[i]
        data_train_count += 1

# Kiểm tra kích thước trước khi huấn luyện
print("Shape of X_train:", X_train.shape)
print("Shape of Y_train:", Y_train.shape)
print("Shape of X_test:", data_test.shape)
print("Shape of Y_test:", Y_test.shape)

# Huấn luyện mô hình Naïve Bayes
model_sk = GaussianNB(priors=None)
model_sk.fit(X_train, Y_train)

# Dự đoán và đánh giá
Y_pred = model_sk.predict(data_test)

print("Accuracy score is:", accuracy_score(Y_test, Y_pred))
print("Precision score is:", precision_score(Y_test, Y_pred, pos_label=4))
print("Recall score is:", recall_score(Y_test, Y_pred, pos_label=4))


Number of samples read: 699
Shape of the dataset: (699, 9)
Number of labels: 699
Shape of X_train: (579, 9)
Shape of Y_train: (579,)
Shape of X_test: (120, 9)
Shape of Y_test: (120,)
Accuracy score is: 0.9333333333333333
Precision score is: 0.8636363636363636
Recall score is: 0.95
